<a href="https://colab.research.google.com/github/seanpaz478/USDAAI521Team1FinalProject/blob/main/AAI_521FinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import kagglehub
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# Getting dataset using Kaggle API
path = kagglehub.dataset_download("andrewmvd/lung-and-colon-cancer-histopathological-images")

In [ ]:
# Loading and preprocessing the dataset
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    horizontal_flip=True
)

train_data = datagen.flow_from_directory(
    path,
    target_size=(150, 150),
    batch_size=64,
    subset='training',
    class_mode='binary'
)

val_data = datagen.flow_from_directory(
    path,
    target_size=(150, 150),
    batch_size=64,
    subset='validation',
    class_mode='binary'
)

Found 20000 images belonging to 1 classes.
Found 5000 images belonging to 1 classes.


In [ ]:
# Using MobileNet pre-trained model as the base
base_model = tf.keras.applications.MobileNetV2(input_shape=(150, 150, 3),
                                               include_top=False,
                                               weights='imagenet'
)

base_model.trainable = False

# Building CNN off of pre-trained model
model = Sequential([
    base_model,
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

<ipython-input-15-85d4888502db>:2: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = tf.keras.applications.MobileNetV2(input_shape=(150, 150, 3),


In [ ]:
# Implementing early stopping to stop training when validation accuracy stops improving
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=3,
    restore_best_weights=True
)
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_data,
                    epochs=50,
                    validation_data=val_data,
                    callbacks=[early_stopping])

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


313/313 ━━━━━━━━━━━━━━━━━━━━ 559s 2s/step - accuracy: 0.9906 - loss: 0.0228 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 552s 2s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 542s 2s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 540s 2s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00


In [ ]:
val_preds = model.predict(val_data)
print(classification_report(val_data.classes, np.argmax(val_preds, axis=1)))

79/79 ━━━━━━━━━━━━━━━━━━━━ 109s 1s/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5000

    accuracy                           1.00      5000
   macro avg       1.00      1.00      1.00      5000
weighted avg       1.00      1.00      1.00      5000



In [ ]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224 (Functional)    │ (None, 5, 5, 1280)          │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 32000)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │       4,096,128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,546,757 (55.49 MB)

 Trainable params: 4,096,257 (15.63 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

 Optimizer params: 8,192,516 (31.25 MB)

In [ ]:
_ = model.predict(train_data[0])

# Using Grad-CAM to visualize model behavior
def grad_cam(model, img_array, layer_name='dense_3'):
    grad_model = tf.keras.models.Model(
        inputs=[model.inputs],
        outputs=[model.get_layer(layer_name).output, model.output]
    )
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(np.array([img_array]))
        loss = predictions[:, 0]

    grads = tape.gradient(loss, conv_outputs)
    weights = tf.reduce_mean(grads, axis=(0, 1))

    cam = tf.reduce_sum(weights * conv_outputs, axis=-1)
    cam = tf.nn.relu(cam)
    return cam[0]

def overlay_cam_on_image(image, cam, alpha=0.4):
    cam_resized = tf.image.resize(cam, image.shape[:2]).numpy()
    cam_resized = (cam_resized - cam_resized.min()) / (cam_resized.max() - cam_resized.min())
    heatmap = plt.cm.jet(cam_resized)[..., :3]
    overlay = (image * (1 - alpha)) + (heatmap * alpha).astype(np.uint8)
    return overlay



2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 924ms/step


In [ ]:
iterator = iter(train_data)
for i in range(5):
    # Geting the image and label
    img, label = next(iterator)
    img = img[0]
    label = label[0]

    # Preprocessing image
    img_array = np.expand_dims(img, axis=0)

    # Generating Grad-CAM heatmap
    cam = grad_cam(model, img_array, layer_name="dense_3")

    # Overlaying Grad-CAM on original image
    overlay = overlay_cam_on_image((img * 255).astype(np.uint8), cam)

    # Plotting results
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.title(f"Original Image (Label: {label})")
    plt.imshow(img)
    plt.axis("off")

    plt.subplot(1, 2, 2)
    plt.title("Grad-CAM Overlay")
    plt.imshow(overlay)
    plt.axis("off")

    plt.tight_layout()
    plt.show()

ValueError: The layer sequential_1 has never been called and thus has no defined output.